In [10]:
#import
from Crypto.Cipher import PKCS1_v1_5
from socket import*
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
import datetime
from Crypto.Cipher import DES
import pickle
import pyodbc
from configparser import ConfigParser
import hashlib
 


    
    
def check_data():

    configur = ConfigParser()
    configur.read('imp_config.ini')

    server_name = configur.get('database', 'server_name')
    database_name = configur.get('database', 'database_name')
    connection_string = 'DRIVER={SQL Server};SERVER=' + server_name + ';DATABASE=' + database_name
    connection = pyodbc.connect(connection_string)
    cursor = connection.cursor()

    response = """
    select sum(summ)+sum(life_time_m) from credits as a1 
    left join clients as a2 on a1.clients_id=a2.id
    left join cr_type as a3 on a1.cr_type_id=a3.id
    left join employers as a4 on a1.employers_id=a4.id
    left join office as a5 on a4.office_id=a5.id
    """
    chek=0
    cursor.execute(response)
    for r in cursor:
        chek=r[0]
        print('Проверочная сумма =' + str(r[0]))
    connection.commit()
    connection.close()
    return chek
    
def connect(): 
    

    serv_sock.bind((ip, int(port)))
    print('Подключение произведено')
    serv_sock.listen(1)
    client_conn, client_addr = serv_sock.accept()
    print('Соединение:', client_addr)
    serv_rsa_key = RSA.generate(1024)
    exported_key = serv_rsa_key.publickey()
    exported_key=exported_key.export_key()
    print('Открытый ключ (md5 хеш): ' + str(hashlib.md5(exported_key).hexdigest()))
    data_msg = pickle.dumps(exported_key)
    client_conn.send(data_msg)
    print('Открытый ключ отправлен')
    return client_conn,serv_rsa_key

def recv_key(client_conn,serv_rsa_key):
    
    data_msg = client_conn.recv(100000)
    enc_msg = pickle.loads(data_msg)
    print('Ключ симметричного шифрования получен:' + str(enc_msg))
    serv_rsa_cipher = PKCS1_OAEP.new(serv_rsa_key)
    des_key = serv_rsa_cipher.decrypt(enc_msg)
    print('Ключ (md5 хеш):'+  str(hashlib.md5(des_key).hexdigest()))
    msg = 'Y'
    data_msg = pickle.dumps(msg.encode('utf-8'))
    client_conn.send(data_msg)
    serv_des_cipher = DES.new(des_key, DES.MODE_ECB)
    return serv_des_cipher
    
    
def load_data(serv_des_cipher,client_conn):
    
    server_name = configur.get('database','server_name')
    database_name = configur.get('database','database_name')
    connection_string = 'DRIVER={SQL Server};SERVER=' + server_name + ';DATABASE=' + database_name 
    connection = pyodbc.connect(connection_string)
    cursor = connection.cursor()
    load_directory(cursor,client_conn,serv_des_cipher)
    load_directory(cursor,client_conn,serv_des_cipher)
    load_directory(cursor,client_conn,serv_des_cipher)
    load_directory_2(cursor,client_conn,serv_des_cipher)
    load_directory_3(cursor,client_conn,serv_des_cipher)
    
    connection.commit()
    connection.close()
    
    chek_v=check_data()
    data_msg = client_conn.recv(100000)
    enc_msg = pickle.loads(data_msg)
    print('Получено заишфрованное сообщение: ' + str(enc_msg))
    dec_msg = serv_des_cipher.decrypt(enc_msg)
    data_ = str(dec_msg.decode('utf-8')).strip()
    print('Сообщение расшифровано: ' + str(dec_msg.decode('utf-8')))
    if (int(data_)==int(chek_v)):
        print('Импорт завершен успешно')
    else:
        print('Возникла ошибка')
    msg = 'Y'
    data_msg = pickle.dumps(msg.encode('utf-8'))
    client_conn.send(data_msg)
    
def load_directory(cursor,client_conn,serv_des_cipher):
    data_msg = client_conn.recv(100000)
    enc_msg = pickle.loads(data_msg)
    print('Получено заишфрованное сообщение: ' + str(enc_msg))
    dec_msg = serv_des_cipher.decrypt(enc_msg)
    data_ = str(dec_msg.decode('utf-8')).strip()
    print('Сообщение расшифровано: ' + str(dec_msg.decode('utf-8')))
    cursor.execute(data_)
    msg = 'Y'
    data_msg = pickle.dumps(msg.encode('utf-8'))
    client_conn.send(data_msg)
    
def load_directory_2(cursor,client_conn,serv_des_cipher):
    data_msg = client_conn.recv(100000)
    enc_msg = pickle.loads(data_msg)
    print('Получено заишфрованное сообщение: ' + str(enc_msg))
    dec_msg = serv_des_cipher.decrypt(enc_msg)
    data_ = str(dec_msg.decode('utf-8')).strip().split("|")
    print('Сообщение расшифровано: ' + str(dec_msg.decode('utf-8')))
    index=""
    for i in range(len(data_)-1):
        if (i%2==0):
            print(data_[i])
            cursor.execute(str(data_[i]))
            index=cursor.fetchone()
        if (i%2==1):
            cursor.execute(data_[i]+" , "+str(index[0])+")")
    msg = 'Countries has been caugth'
    data_msg = pickle.dumps(msg.encode('utf-8'))
    client_conn.send(data_msg)

    
    
def load_directory_3(cursor,client_conn,serv_des_cipher):
    data_msg = client_conn.recv(100000)
    enc_msg = pickle.loads(data_msg)
    print('Получено заишфрованное сообщение: ' + str(enc_msg))
    dec_msg = serv_des_cipher.decrypt(enc_msg)
    data_ = str(dec_msg.decode('utf-8')).strip().split("|")
    print('Сообщение расшифровано: ' + str(dec_msg.decode('utf-8')))
    index=""
    index1=""
    index2=""
    for i in range(len(data_)-1):
        if (i%4==0):
            cursor.execute(str(data_[i]))
            index=cursor.fetchone()
        if (i%4==1):
            cursor.execute(str(data_[i]))
            index1=cursor.fetchone()
        if (i%4==2):
            cursor.execute(str(data_[i]))
            index2=cursor.fetchone()
        if (i%4==3):
            cursor.execute(data_[i]+" , "+str(index[0])+" , "+str(index1[0])+" , "+str(index2[0])+")")
    msg = 'Y'
    data_msg = pickle.dumps(msg.encode('utf-8'))
    client_conn.send(data_msg)
    
configur = ConfigParser()
configur.read('imp_config.ini')
ip = configur.get('network','ip')
port = configur.get('network','port')
serv_sock = socket(AF_INET, SOCK_STREAM)
client_conn,serv_rsa_key=connect()
serv_des_cipher=recv_key(client_conn,serv_rsa_key)
load_data(serv_des_cipher,client_conn)
print('End')



Подключение произведено
Соединение: ('127.0.0.1', 55477)
Открытый ключ (md5 хеш): 84c2827c088acd8e233508339f40efc0
Открытый ключ отправлен
Ключ симметричного шифрования получен:b'?E\x13Q\xe0*\xac\x88\xf9\x93,\xa7\xf9\x86:&g\x1b\x82\x18\xb5&0\r\xfc:O\x83\x9b<k\'\xd2 \x10\xfe,=~R\xf7?\xae\xd10\x0cb\x0f3\x8c\xf6\xbe\xcf\x9d_\xb8\xf4\x13s\xdee\x05\xe3\x98\xf0\x03\x00\xb0%Y\x9ec\xa1\xce\xe1\xb8i\x80\xeb\xbc5M\xf5\x00\xf1\xa6\xd3\x0b"K\x93\x8d\xbe$\x8a-\xccgK\x1a\t\xc5<\x91\xd6\x8a\x1a\xc2\xecW\x07\x8f\x8b\x11=\x82\xdcR>k\xf8\xfa`\xf7{\'y\x89'
Ключ (md5 хеш):ba340fd0a69f153a103476d4dbb79751
Получено заишфрованное сообщение: b'\xe5\t\xfa9\xe3,\xfc|\xf7\xdc0\xaaA|\xbb\xb8\xc8\ns\x06M\r\xa1\xf7`\xc7\xc7X\xdaJ\xd9\xe9\x94L\x82\x04J\x1ag7(\x82\xc0\x0c\xf2\xbf\xdc\x93\x1f\xe8\xbc\xfb\xb5e\xc0\x1af\x86\\\x82\x99\xaaX\xef\xeb6dC6\xb2.\xa6 Y\xc0m\xe8\xea\x907d\x11\xb6#\xe3Y\x92\xdd\xe0\xb5\xe6\xa9\xdcl\xb4\xf6\x9d\xde+\xf8\xe9\x08\xd6-\x06\xd3\xc3\x92R\xcd\xed \xc9\xcb@\x1bLxM9\x17\x04\xc8e\xce\x14\x

In [1]:
pip install pycryptodome --upgrade

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade pip

  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip-upgrad 

Note: you may need to restart the kernel to use updated packages.



Usage:   
  C:\Games\ANACONDA\python.exe -m pip <command> [options]

no such option: -u


In [6]:


configur = ConfigParser()
configur.read('imp_config.ini')

server_name = configur.get('database', 'server_name')
database_name = configur.get('database', 'database_name')
connection_string = 'DRIVER={SQL Server};SERVER=' + server_name + ';DATABASE=' + database_name
connection = pyodbc.connect(connection_string)
cursor = connection.cursor()

response = """
select sum(summ)+sum(life_time_m) from credits as a1 
left join clients as a2 on a1.clients_id=a2.id
left join cr_type as a3 on a1.cr_type_id=a3.id
left join employers as a4 on a1.employers_id=a4.id
left join office as a5 on a4.office_id=a5.id
"""
cursor.execute(response)
for r in cursor:
    print(r[0])

connection.close()

print('end')

1500350.00
end
